# Clean Data

#### Imports

In [1]:
# Import pandas and numpy
import pandas as pd
import numpy as np


#### Get csv file

In [2]:
# Import the csv module
import csv

# Import and read the file with a with statement
with open('games.csv', 'r') as file:
    reader = csv.reader(file)

df = pd.read_csv('games.csv', sep=',')

In [3]:
# Drop Columns
df = df.drop(['Unnamed: 10'], axis=1)
df = df.drop(['metacritic'], axis=1)


In [4]:
df_genre = df['genre'].str.get_dummies('-')
df = pd.concat([df[:], df_genre[:]], axis=1)


df_tags = df['tags'].str.get_dummies('-')
df = pd.concat([df[:], df_tags[:]], axis=1)


df_platforms = df['platforms'].str.get_dummies('-')
df = pd.concat([df[:], df_platforms[:]], axis=1)

#df_series = df['series'].str.split('-')



In [5]:
df = df.drop(['genre'], axis=1)
df = df.drop(['tags'], axis=1)
df = df.drop(['platforms'], axis=1)
df = df.drop(['series'], axis=1)
df

,id,name,released,rating,playtime,Action,Adventure,Arcade,Board Games,Card,...,Web,Wii,Wii U,Xbox,Xbox 360,Xbox One,Xbox Series S/X,bit,iOS,macOS
0,12341.0,King Arthur's Gold,2013-11-04,2.90,2.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3498.0,Grand Theft Auto V,2013-09-17,4.47,73.0,1,1,0,0,0,...,0,0,0,0,1,1,1,0,0,0
2,3328.0,The Witcher 3: Wild Hunt,2015-05-18,4.66,46.0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,4200.0,Portal 2,2011-04-18,4.62,11.0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
4,5286.0,Tomb Raider (2013),2013-03-05,4.05,10.0,1,1,0,0,0,...,0,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9955,43811.0,Castle of no Escape,2017-06-22,1.33,3.0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9956,43426.0,Dying Light: The Following,2016-02-09,3.96,0.0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9957,39108.0,Fallout 3: Operation Anchorage,2009-01-27,3.65,0.0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9958,38140.0,Soldier of Fortune: Payback,2007-11-12,2.51,1.0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
df = df.dropna()

In [7]:
df['rating'] = (df['rating'] - df['rating'].mean())/df['rating'].std()

C:\Users\andre\AppData\Local\Temp\ipykernel_4160\3992767191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = (df['rating'] - df['rating'].mean())/df['rating'].std()


In [8]:
df['playtime'] = (df['playtime'] - df['playtime'].mean())/df['playtime'].std()
print(df['playtime'])

0      -0.136826
1       5.106939
2       3.112831
3       0.527876
4       0.454021
          ...   
9955   -0.062970
9956   -0.284538
9957   -0.284538
9958   -0.210682
9959   -0.284538
Name: playtime, Length: 9804, dtype: float64


C:\Users\andre\AppData\Local\Temp\ipykernel_4160\1785298778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['playtime'] = (df['playtime'] - df['playtime'].mean())/df['playtime'].std()


In [33]:
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model 
        to generalize and improves the interpretability of the model.

    Inputs: 
        x: features dataframe
        threshold: features with correlations greater than this value are removed

    Output: 
        dataframe that contains only the non-highly-collinear features
    '''

    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns=drops)

    return x

In [35]:
remove_collinear_features(df.drop(['name','released'], axis=1), .9)

In [10]:
corr = df.drop(['name','released'], axis=1).corr()

In [11]:
mask = np.where(corr > 0.8) # create a mask for the condition
for i,j in zip(*mask): # loop over the indices of the mask
    if i != j: # exclude the diagonal elements
        print(f"{corr.index[i]} and {corr.columns[j]}: {corr.iloc[i,j]}") # print the 

 NES and Virtual Console : 1.0
007 and espionage: 1.0
007 and james bond: 1.0
16 and bit: 0.9486348698977463
1930s and 30s: 1.0
1960s and germany: 1.0
1970s and cleaning: 1.0
1970s and latin: 1.0
1970s and revolution: 1.0
1970s and sunset: 1.0
1984 and buddy: 1.0
1st and dark: 1.0
1st and dark sci: 1.0
2017 and indiecade: 1.0
2D Platformer and платформер: 0.9716758387143342
3 and gpt: 1.0
30s and 1930s: 1.0
3d and adventurer: 1.0
3d and dwarves: 1.0
3d and elves: 1.0
3d and enroth: 1.0
3d and goblins: 1.0
3d and magic: 1.0
3d and might: 1.0
3d and might and magic: 1.0
3d and xeen: 1.0
3rd and Person Perspective: 0.9926761926049449
4 and arcade: 1.0
4 and brawl: 1.0
4 and brawling: 1.0
4 and cage: 1.0
4 and entertainment: 1.0
4 and four: 1.0
4 and mayhem: 1.0
4 and mode: 1.0
4 and royal: 1.0
4 and royal rumble: 1.0
4 and rumble: 1.0
4 and steel: 1.0
4 and stone cold steve austin: 1.0
4 and tournament: 1.0
4 and unreal: 1.0
4 and way: 1.0
4 and wrestle: 1.0
4 and wrestling: 1.0
4 and wwe

In [13]:
df.to_csv("clean_data.csv", index=False)

,id,name,released,rating,playtime,Action,Adventure,Arcade,Board Games,Card,...,Web,Wii,Wii U,Xbox,Xbox 360,Xbox One,Xbox Series S/X,bit,iOS,macOS
0,12341.0,King Arthur's Gold,2013-11-04,-0.084234,-0.136826,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3498.0,Grand Theft Auto V,2013-09-17,1.335551,5.106939,1,1,0,0,0,...,0,0,0,0,1,1,1,0,0,0
2,3328.0,The Witcher 3: Wild Hunt,2015-05-18,1.507373,3.112831,1,1,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,4200.0,Portal 2,2011-04-18,1.471200,0.527876,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
4,5286.0,Tomb Raider (2013),2013-03-05,0.955736,0.454021,1,1,0,0,0,...,0,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9955,43811.0,Castle of no Escape,2017-06-22,-1.504019,-0.062970,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9956,43426.0,Dying Light: The Following,2016-02-09,0.874347,-0.284538,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9957,39108.0,Fallout 3: Operation Anchorage,2009-01-27,0.594008,-0.284538,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9958,38140.0,Soldier of Fortune: Payback,2007-11-12,-0.436919,-0.210682,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#### Close file

In [ ]:
file.close()